# Value-based agent using PyTorch

In [1]:
import copy
import time
import torch
import imageio
import gymnasium as gym
import matplotlib.pyplot as plt

from agents.torch_value_vision import VisionDeepQ

## Tetris environment

|        | TYPE                   | VALUES          | DESCRIPTION                                                                                                |
|--------|------------------------|-----------------|------------------------------------------------------------------------------------------------------------|
| Action Space | ndarray<br/>(1,) | {0, 1, 2, 3, 4} | Action to manipulate the current tile.<br/>0: No action<br/>1: Rotate<br/>2: Right<br/>3: Left<br/>4: Down |
| Observation Space | ndarray<br/>(210,160,) | <0, 255> | The game screen.                                                                                           |
| Reward |  | float | Reward given when a row is filled.<br/>Single: 50<br/>Double: 150<br/>Triple: 400<br/>Quadruple: 900       |
| Termination |  | boolean | The game ends when the pieces stack up to the top of the playing field.                                    |

In [2]:
environment = gym.make('ALE/Tetris-v5', render_mode="rgb_array", obs_type="grayscale")

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


### Training

NOTE:

The mini-batch value represents how many _full_ games to train on from the agents memor. 
Therefore, an additional parameter `TRAIN_EVERY` is introduced to control how often the agent 
should learn from the memorized games. Although, even with this parameter, the training time 
increases significantly as the agent improves and has to learn from longer and longer games.

In [3]:
GAMES = 50

GAMMA = 0.99

EXPLORATION_RATE = 1.0
EXPLORATION_DECAY = 0.995
EXPLORATION_MIN = 0.01

MINIBATCH = 64
TRAIN_EVERY = 10

MEMORY = 1500
RESET_Q_EVERY = 250

NETWORK = {"input_channels": 1, "outputs": 5, 
           "channels": [4, 8],
           "kernels": [(3, 3), (3, 3), (5, 5)]
           }
OPTIMIZER = {"optim": torch.optim.RMSprop, "lr": 0.00025}

In [4]:
value_agent = VisionDeepQ(
    network=NETWORK, optimizer=OPTIMIZER,
    gamma=GAMMA, batch_size=MINIBATCH, memory=MEMORY,
    exploration_rate=EXPLORATION_RATE, exploration_decay=EXPLORATION_DECAY, exploration_min=EXPLORATION_MIN
)

_value_agent = copy.deepcopy(value_agent)

In [5]:
checkpoint = GAMES // 10
metrics = {
    "steps": torch.zeros(GAMES),
    "losses": torch.zeros(GAMES // TRAIN_EVERY),
    "exploration": torch.zeros(GAMES)
}

In [6]:
start = time.time()
for game in range(1, GAMES + 1):
    
    state = torch.tensor(environment.reset()[0], dtype=torch.float32).unsqueeze(0)  # noqa
    terminated = truncated = False
    
    # LEARNING FROM GAME
    # --------------------------------------------------
    
    steps = 0
    while not (terminated or truncated):
        steps += 1
        action = value_agent.action(state)
        new_state, reward, terminated, truncated, _ = environment.step(action.item())
        new_state = torch.tensor(new_state, dtype=torch.float32).unsqueeze(0)

        value_agent.remember(state, action, new_state, torch.tensor([reward]))
        state = new_state
    value_agent.memorize(steps)
    
    if game % TRAIN_EVERY == 0:
        loss = value_agent.learn(network=_value_agent)
        metrics["losses"][game // TRAIN_EVERY - 1] = loss
    
    if game % RESET_Q_EVERY == 0:
        _value_agent.load_state_dict(value_agent.state_dict())

    # METRICS
    # --------------------------------------------------
    
    metrics["steps"][game-1] = steps
    metrics["exploration"][game-1] = value_agent.explore["rate"]
    
    if game % checkpoint == 0 or game == GAMES:
        _mean_steps = metrics["steps"][max(0, game-checkpoint-1):game-1].mean()
        _mean_loss = metrics["losses"][max(0, (game-checkpoint-1) // TRAIN_EVERY):game // TRAIN_EVERY].mean()
        
        print(f"Game {game:>6} {int(game/GAMES * 100):>16} % \n"
              f"{'-'*30} \n"
              f" > Average steps: {int(_mean_steps):>12} \n"
              f" > Average loss: {_mean_loss:>13.4f} \n ")
        
print(f"Total training time: {time.time()-start:.2f} seconds")

Game      5               10 % 
------------------------------ 
 > Average steps:          522 
 > Average loss:           nan 
 


RuntimeError: Given groups=1, weight of size [4, 1, 3, 3], expected input[1, 5313, 210, 160] to have 1 channels, but got 5313 channels instead

#### Visualisation

In [ ]:
def moving_average(data, window_size=50):
    """Compute moving average with given window size of the data."""
    half_window = window_size // 2
    return [(data[max(0, i-half_window):min(len(data), i+half_window)]).mean() 
            for i in range(len(data))]

steps = moving_average(metrics["steps"])
losses = moving_average(metrics["losses"])

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
fig.suptitle("Deep Q-Learning vision agent")

ax[0].axhline(y=500, color="red", linestyle="dotted", linewidth=1)
ax[0].plot(steps, color="black", linewidth=1)
ax[0].set_xticks([])
ax[0].set_title("Average steps per game")

ax[1].plot(torch.linspace(0, GAMES, len(losses)), losses, color="black", linewidth=1)
ax[1].set_xlabel("Game nr.")
ax[1].set_title("Average loss")

ax_2 = ax[1].twinx()
ax_2.plot(metrics["exploration"], color="gray", linewidth=0.5)
ax_2.set_ylabel("Exploration rate")
ax_2.yaxis.label.set_color('gray')
ax_2.tick_params(axis='y', colors='gray')

for i in range(0, GAMES, GAMES // 10):
    ax[0].axvline(x=i, color='gray', linewidth=0.5)
    ax[1].axvline(x=i, color='gray', linewidth=0.5)

plt.savefig("./static/images/torch-dqn-tetris.png")
plt.show()

##### In action

In [ ]:
state = torch.tensor(environment.reset()[0], dtype=torch.float32)

images = []
terminated = truncated = False
while not (terminated or truncated):
    action = torch.argmax(value_agent(state)).item()
    
    state, reward, terminated, truncated, _ = environment.step(action)
    state = torch.tensor(state, dtype=torch.float32)

    images.append(environment.render())
_ = imageio.mimsave('./static/images/torch-dqn-tetris.gif', images, duration=25)

<img src="./static/images/torch-dqn-tetris.gif" width="1000" height="1000" />

In [ ]:
environment.close()